In [1]:
# import required libraries
import numpy as np
import pandas as pd
from pandas_datareader import data
import matplotlib.pyplot as plt
import yaml
import os

In [2]:
# Calling the yaml file
with open("./datapull.yml", 'r') as f:
    params = yaml.safe_load(f)

In [3]:
# Creating a directory to save pulled datasets
directory='./Google_Finance/'
if not os.path.exists(directory):
        os.makedirs(directory)

In [4]:
# Creating and saving the dataset to specified folder

# enter start date and end date
start_date = params['start_date']
end_date = params['end_date']

# Tickers correspond to Companies
# Enter Tickers here
tickers = params['tickers']
for ticker in tickers:
    try:
        count=1

        panel_data = data.DataReader(ticker, 'yahoo', start_date, end_date)
        panel_data=panel_data.reset_index()

        panel_data['Date'] = pd.to_datetime(panel_data['Date'], format='%Y-%m-%d')
        panel_data['month']= pd.DatetimeIndex(panel_data['Date']).month
        panel_data['year'] = pd.DatetimeIndex(panel_data['Date']).year
        panel_data = panel_data.drop(['Date','Volume'],axis=1).groupby(['year','month']).agg({'High' : 'mean','Low': 'mean',
                                                                                              'Open': 'mean', 'Close':'mean'})
        panel_data =panel_data.reset_index()
        panel_data.columns = ['year'+ticker,'month'+ticker,'High'+ticker,'Low'+ticker,'Open'+ticker,'Close'+ticker]

        #print(panel_data.head())

        if count == 1:                                                                                      
            df = pd.DataFrame(index=np.arange(len(panel_data)),columns=['year','month'])
            df = df.fillna(0)
            df['month'] = panel_data['month'+ticker]
            df['year'] = panel_data['year'+ticker]
            count=count+1

        #print(df.head())
        panel_data=panel_data.drop(['year'+ticker,'month'+ticker],axis=1)
        df = pd.merge(df,panel_data, left_index = True, right_index = True)
        #df = pd.concat([df.reset_index(drop=True), panel_data], axis=1)
        df.to_csv(directory+ticker+"_finance.csv",index=False)
        #print(df.head())
    except:
        k=1
    
    

In [5]:
# Reading the files and merging it into a single dataframe
import os
list_1 =os.listdir(directory)

x= pd.read_csv(directory+"^IXIC_finance.csv")

df = pd.DataFrame(index=np.arange(len(panel_data)),columns=['year','month'])
df = df.fillna(0)
df['month'] = x['month']
df['year'] = x['year']
for i in list_1:
    df=pd.merge(df,pd.read_csv(directory+str(i)),how='outer',on=['year','month'])
    

df.to_csv(directory+"Finance_Final_data.csv",index=False)   